<a target="_blank" href="https://colab.research.google.com/github/okareo-ai/okareo-python-sdk/blob/main/examples/generation_eval.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Welcome to Okareo!

Get your API token from [https://app.okareo.com/](https://app.okareo.com/) and set it in the cell below. 👇
   (Note: You will also need an OpenAI key.)



In [ ]:
OKAREO_API_KEY = "<YOUR-OKAREO-API-TOKEN>"
OPENAI_API_KEY = "<YOUR-OPENAI-API-TOKEN>"

In [ ]:
%pip install okareo
%pip install openai

For the final step in a RAG pipeline, generation, we're going to set up a summarization task for OpenAI's GPT 3.5 Turbo model. The goal of the task is for the model to summarize 10 documents about different aspects of our example web business, WebBizz. The model will be evaluated on the coherence, consistency, fluency, and relevance of its summarizations.

The task will have three parts:

1. A database of documents
2. A method for prompting the model for summarizations
3. An evaluation of the summarizations

## Document database

In [ ]:
# Import libraries
import os
import tempfile

# Import Okareo libraries
from okareo import Okareo
from okareo_api_client.models.test_run_type import TestRunType

# Create an instance of the Okareo client
okareo = Okareo(OKAREO_API_KEY)

# Load documents from Okareo's GitHub repository
webbizz_articles = os.popen('curl https://raw.githubusercontent.com/okareo-ai/okareo-python-sdk/main/examples/webbizz_10_articles.jsonl').read()

# Save the documents to a temporary file
temp_dir = tempfile.gettempdir()
file_path = os.path.join(temp_dir, "webbizz_10_articles.jsonl")
with open(file_path, "w+") as file:
    lines = webbizz_articles.split('\n')
    # Limit the number of documents to 3
    for i in range(3):
        file.write(f"{lines[i]}\n")

# Upload a scenario set to Okareo with the documents
scenario = okareo.upload_scenario_set(file_path=file_path, scenario_name="Webbizz Articles Scenario")

# Clean up tmp file
os.remove(file_path)

## Summarization Model

In [ ]:
# Import libraries
from openai import OpenAI
from datetime import datetime

# Import Okareo's handler for OpenAI models
from okareo.model_under_test import OpenAIModel

# Create an instance of the OpenAI client
client = OpenAI(api_key=OPENAI_API_KEY)

# Define a function to call the OpenAI API
# This function will be used to query the specified OpenAI model
def get_turbo_summary(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=500):
  response = client.chat.completions.create(
    model=model,
    messages=messages,
    temperature=temperature, 
    max_tokens=max_tokens,
  )
  return response.choices[0].message.content

# Define a template for the user prompt
USER_PROMPT_TEMPLATE = "{scenario_input}"

# Define a template to prompt the model to provide a summary
SUMMARIZATION_CONTEXT_TEMPLATE = """
You will be provided with text.
Summarize the text in 1 simple sentence.
"""

# Create an instance of the OpenAIModel class
# This class is used to interact with the OpenAI model using user and system prompts
openai_model = OpenAIModel(
        model_id="gpt-3.5-turbo",
        temperature=0,
        system_prompt_template=SUMMARIZATION_CONTEXT_TEMPLATE,
        user_prompt_template=USER_PROMPT_TEMPLATE,
    )

# Define the name of the model with the current timestamp
mut_name=f"OpenAI Summarization Model - {datetime.now().strftime('%m-%d %H:%M:%S')}"

# Register the model to use in a test run
model_under_test = okareo.register_model(
    name=mut_name,
    model=openai_model,
)

## Evaluation

In [ ]:
# Create a name for the evaluation with the current timestamp
eval_name = f"Summarization Run - {datetime.now().strftime('%m-%d %H:%M:%S')}"

# Perform a test run using the scenario set and the summarization model
evaluation = model_under_test.run_test(
    name=eval_name,
    scenario=scenario,
    api_key=OPENAI_API_KEY,
    test_run_type=TestRunType.NL_GENERATION, # specify that we are testing a natural language generation model
    calculate_metrics=True,
    # define the metrics to calculate
    checks=['coherence_summary', 'consistency_summary', 'fluency_summary', 'relevance_summary']
)

# Generate a link back to Okareo for evaluation visualization
print(f"See results in Okareo: {evaluation.app_link}")